In [2]:
import copy
import tqdm

In [11]:
charts = []
contigs = []

with open("test.txt", "r") as file:
    for l in file.readlines():
        chart, contig = l.strip().split(' ')
        charts.append(chart)
        contigs.append(list(map(int, contig.split(','))))

In [4]:
def add_hash(forest):
    f = copy.deepcopy(forest)
    for tree in f:
        tree[-1] += 1
    return f

def add_dot(forest):
    f = copy.deepcopy(forest)
    # If length == 1, then return [0,0] not [0].
    if forest == [[0]]:
        return [[0, 0]]
    for tree in f:
        if tree[-1] != 0:
            tree.append(0)
    return f

In [5]:
def is_possible(tree, target):
    # if len(tree) > 1 and tree[0] == 0:
    #     tree = tree[1:]
    if len(tree) > len(target) + 1:
        return False
    if len(tree) == len(target) + 1 and tree[-1] != 0:
        return False
    for i in range(len(tree) - 1):
        if tree[i] != target[i]:
            return False
    if len(tree) <= len(target):
        if tree[len(tree)-1] > target[len(tree)-1]:
            return False
    
    return True

def is_equal(tree, target):
    # tree == target, or tree == target + [0]
    return tree == target or tree == target + [0]

    # if len(tree) < len(target):
    #     return False

    # for i in range(len(target)):
    #     if tree[i] != target[i]:
    #         return False
     
    # if len(tree) > len(target) and tree[-1] != 0:
    #     return False
    # return True

In [6]:
def combine_contigs(c1, c2, w1, w2):
    dot = [*c1, *c2]
    hash = [*c1[:-1], c1[-1] + 1 + c2[0], *c2[1:]]
    return dot, hash, w1 * w2

In [7]:
def get_num_possibilities(chart, target):

    forest = [[0]]

    for symbol in chart:
        if symbol == '#':
            forest = add_hash(forest)
        if symbol == '.':
            forest = add_dot(forest)
        if symbol == '?':
            #print(add_hash(forest.deepcopy()))
            #print(add_dot(forest.deepcopy()))
            forest = [*add_hash(forest), *add_dot(forest)]

        #print(forest)
        
        #forest = [t for t in forest if is_possible(t, target)]

    # len([t for t in forest if is_equal(t, target)]), 
    return forest # and is_possible(t, target)

In [8]:
def array_to_str(contig):
    return ','.join([str(d) for d in contig])
def str_to_array(string):
    return [int(s) for s in string.split(',')]

In [9]:
def dot_join(c1, c2):
    return [a for a in [*c1, *c2[:-1]] if a != 0] + [c2[-1]]

def hash_join(c1, c2):
    [*c1[:-1], c1[-1] + 1 + c2[0], *c2[1:]]

In [10]:
def unfolded_possiblities(chart, target):
    folded_list = get_num_possibilities(chart, target)
    folded_count = {}
    for t_array in folded_list:
        t = array_to_str(t_array)
        if t in folded_count.keys():
            folded_count[t] += 1
        else:
            folded_count[t] = 1
    
    unfolded_target = target * 5

    forest = {}
    
    for tree_str, count in folded_count.items():
        tree_arr = str_to_array(tree_str)
        if len(tree_arr) > 1 and tree_arr[0] == 0:
            tree_arr = tree_arr[1:]
        if is_possible(tree_arr, unfolded_target):
            new_tree_str = array_to_str(tree_arr)
            if new_tree_str in forest.keys():
                forest[new_tree_str] += count
            else:
                forest[new_tree_str] = count

    for i in range(4): #0,1,2,3

        extended_forest = {}

        for base, base_count in forest.items():
            for extention, extention_count in folded_count.items():
                c1 = str_to_array(base)
                c2 = str_to_array(extention)
                dot = [a for a in [*c1, *c2[:-1]] if a != 0] + [c2[-1]]
                # ^ could this be introducing 0?
                hash = [*c1[:-1], c1[-1] + 1 + c2[0], *c2[1:]]
                for tree in [dot, hash]:
                    if is_possible(tree, unfolded_target): 
                        t = array_to_str(tree)
                        if t in extended_forest.keys():
                            extended_forest[t] += base_count * extention_count
                        else:
                            extended_forest[t] = base_count * extention_count

        forest = extended_forest
        # {tree:count for tree, count in extended_forest.items() if is_possible(tree, unfolded_target)}

    n_poss = sum(count for tree, count in forest.items() if is_equal(str_to_array(tree), unfolded_target))
    return forest, n_poss

# print(unfolded_possiblities('?###????????', [3,2,1]))
# print(unfolded_possiblities('????.######..#####.', [1, 6, 5]))
                

In [12]:
total = 0
for i, (chart, contig) in tqdm.tqdm(enumerate(zip(charts, contigs))):
    # print(i)
    #print(chart, contig)
    forest, n_poss = unfolded_possiblities(chart, contig)
    #print(n_poss)
    #print()
    total += n_poss

print(total)

6it [00:00, 234.60it/s]

525152
